<a href="https://colab.research.google.com/github/Mirwais-Farahi/LTA-DataQualityCheck/blob/main/LTADataCollection_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import data_table

data_table.enable_dataframe_formatter()
!pip install koboextractor
!pip install openpyxl
from koboextractor import KoboExtractor

In [ ]:
# Take input from the user for KoBoToolbox API token and asset UID
KOBO_TOKEN = input("Enter your KoBoToolbox API token: ")
asset_uid = input("Enter the asset UID for the form: ")

# Set up the KoBoToolbox API with the token and base URL
kobo = KoboExtractor(KOBO_TOKEN, 'https://eu.kobotoolbox.org/api/v2')

# Fetch data submitted after a specific date
new_data = kobo.get_data(asset_uid, submitted_after='2024-10-06')

# Convert the results directly into a DataFrame
df = pd.DataFrame(new_data['results'])

# Display the number of rows in the DataFrame
print(f"Number of records: {len(df)}")

In [ ]:
df.head()

In [ ]:
row_count = df.shape[0]
print("Number of rows in the DataFrame:", row_count)

In [ ]:
# Set display options
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

In [ ]:
df[["project_code", "assistance"]].drop_duplicates()

In [ ]:
# Initialize lists to hold project codes and intervention types
project_codes_to_count = []
intervention_types_to_count = []

# Function to get user input and add to the lists
def add_to_list():
    print("Enter project codes. Type 'done' when finished:")
    while True:
        project_code = input("Enter a project code (or 'done' to finish): ")
        if project_code.lower() == 'done':
            break
        project_codes_to_count.append(project_code)
        print(f"Added project code '{project_code}' to the list.")

    print("Enter intervention types. Type 'done' when finished:")
    while True:
        intervention_type = input("Enter an intervention type (or 'done' to finish): ")
        if intervention_type.lower() == 'done':
            break
        intervention_types_to_count.append(intervention_type)
        print(f"Added intervention type '{intervention_type}' to the list.")

# Get user input
add_to_list()

# Filter the DataFrame by the specified project codes and intervention types
filtered_df = df[(df['project_code'].isin(project_codes_to_count)) & (df['assistance'].isin(intervention_types_to_count))]

# Group by Province and District and count the number of surveys in each district
result = filtered_df.groupby(['province', 'district']).size().reset_index(name='Total Surveys in District')

# Display the result
result

In [ ]:
filtered_df = filtered_df.dropna(axis=1, how='all')
filtered_df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ask the user for the column name
column_name = input("Enter the column name to plot: ")

# Convert the column to numeric, forcing errors to NaN
filtered_df[column_name] = pd.to_numeric(filtered_df[column_name], errors='coerce')

# Drop rows with NaN values in the column
numeric_df = filtered_df.dropna(subset=[column_name])

# Define the column of interest
numeric_columns = [column_name]

# Draw the box plot
plt.figure(figsize=(12, 8))

# Create the box plot with improved readability
box = plt.boxplot(numeric_df[numeric_columns].values, vert=False, patch_artist=True, labels=numeric_columns)

# Customize the box plot
plt.title(f'Box Plot for {column_name}', fontsize=16)
plt.xlabel(f'Number of {column_name}', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)

# Set colors for the box plot elements
colors = ['#FF9999']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Customize whiskers and medians
for whisker in box['whiskers']:
    whisker.set(color='#7570b3', linewidth=2)
for cap in box['caps']:
    cap.set(color='#7570b3', linewidth=2)
for median in box['medians']:
    median.set(color='orange', linewidth=2)
for flier in box['fliers']:
    flier.set(marker='o', color='#e7298a', alpha=0.5)

plt.show()

In [ ]:
# Describe method
description = numeric_df[column_name].describe()
print(f"Descriptive statistics for '{column_name}':")
print(f"  - Number of entries: {description['count']}")
print(f"  - Average value: {description['mean']}")
print(f"  - Standard deviation: {description['std']}")
print(f"  - Minimum value: {description['min']}")
print(f"  - 25th percentile (25% of values are below this): {description['25%']}")
print(f"  - Median (50% of values are below this): {description['50%']}")
print(f"  - 75th percentile (75% of values are below this): {description['75%']}")
print(f"  - Maximum value: {description['max']}")

# Percentile calculation
percentiles = [0.9, 0.99]  # 90th and 99th percentiles
percentile_values = numeric_df[column_name].quantile(percentiles)
print(f"\n90th Percentiles for '{column_name}':")
print(f"  - 90th percentile (90% of values are below this): {percentile_values[0.9]}")
print(f"  - 99th percentile (99% of values are below this): {percentile_values[0.99]}")


In [ ]:
numeric_df[column_name].hist()
plt.xlabel(f'{column_name} Values Distribution')
plt.ylabel('Frequency')
plt.title(f'Histogram')
plt.show()

In [ ]:
# Calculate the IQR for outlier detection
Q1 = filtered_df[numeric_columns[0]].quantile(0.25)
Q3 = filtered_df[numeric_columns[0]].quantile(0.75)
IQR = Q3 - Q1

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Detect outliers
outliers_df = filtered_df[(filtered_df[numeric_columns[0]] < lower_bound) | (filtered_df[numeric_columns[0]] > upper_bound)]

print(f'Outliers Len: {len(outliers_df)}')

In [ ]:
# Get the index of the specific column
col_index = outliers_df.columns.get_loc(column_name)

# Select columns from 0 to the specific column
selected_columns_outliers_df = outliers_df.iloc[:, :col_index + 1]

selected_columns_outliers_df

In [ ]:
from datetime import datetime
from google.colab import files

# Get current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define the file path with date and time
file_name = f"outliers_{column_name}_{current_time}.xlsx"

# Write DataFrame to an Excel file
selected_columns_outliers_df.to_excel(file_name, index=False)

# Automatically download the file to the local machine
files.download(file_name)

print(f"Data successfully saved and download initiated for {file_name}")

In [ ]:
# Convert 'start' and 'end' columns to datetime with error handling
try:
    filtered_df['start'] = pd.to_datetime(filtered_df['start'], errors='coerce')
    filtered_df['end'] = pd.to_datetime(filtered_df['end'], errors='coerce')
except Exception as e:
    print(f"Error converting to datetime: {e}")

# Calculate the duration in minutes, handling NaT values
filtered_df['duration'] = (filtered_df['end'] - filtered_df['start']).dt.total_seconds() / 60

# Get minimum and maximum duration from the user
try:
    min_duration = float(input("Enter the minimum duration in minutes: "))
    max_duration = float(input("Enter the maximum duration in minutes: "))
except ValueError as e:
    print(f"Invalid input: {e}")
    min_duration = 0
    max_duration = float('inf')

# Find surveys that took between min_duration and max_duration minutes
surveys_with_less_time_df = filtered_df[
    (filtered_df['duration'] > min_duration) & (filtered_df['duration'] < max_duration)
]

print(f'Number of Surveys with less than {min_duration} and more than {max_duration}:', len(surveys_with_less_time_df))

In [ ]:
# Get the index of the specific column
col_index = filtered_df.columns.get_loc('duration')

# Get the list of columns to select
cols_to_select = list(filtered_df.columns[:18])  # First 10 columns
if 'duration' not in cols_to_select:
    cols_to_select.append('duration')  # Add 'duration' if it's not already in the list

# Select columns from the DataFrame
selected_surveys_with_less_time_df = filtered_df[cols_to_select]

# Display the DataFrame with selected columns
selected_surveys_with_less_time_df

In [ ]:
from datetime import datetime
from google.colab import files

# Convert object type columns that should be datetimes
for col in surveys_with_less_time_df.columns:
    if surveys_with_less_time_df[col].dtype == 'object':
        try:
            # Attempt to convert the column to datetime
            surveys_with_less_time_df[col] = pd.to_datetime(surveys_with_less_time_df[col], errors='coerce')
        except Exception as e:
            print(f"Error converting column '{col}' to datetime: {e}")

# Convert timezone-aware datetime columns to timezone-naive
def convert_to_naive(df):
    for col in df.select_dtypes(include=['datetime64[ns, UTC]']).columns:
        if df[col].dt.tz is not None:
            df[col] = df[col].dt.tz_localize(None)
    return df

# Convert timezone-aware datetime columns to timezone-naive
surveys_with_less_time_df = convert_to_naive(surveys_with_less_time_df)

# Get current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define the file path with date and time
file_name = f"surveys_collected_moreThan_{min_duration}_lessThan_{max_duration}_{current_time}.xlsx"

# Write DataFrame to an Excel file
surveys_with_less_time_df.to_excel(file_name, index=False)

# Automatically download the file to the local machine
files.download(file_name)

print(f"Data successfully saved and download initiated for {file_name}")